### 8.1 인스턴스의 문자열 표현식 변형

In [2]:
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self):
        return 'Pair({0.x!r}, {0.y!r})'.format(self)
    def __str__(self):
        return '{0.x!r}, {0.y!r}'.format(self)

In [7]:
p = Pair(3,4)
p

Pair(3, 4)

In [8]:
print (p)

3, 4


### 8.2 문자열 서식화 조절

In [12]:
_formats = {
    'ymd' : '{d.year}-{d.month}-{d.day}',
    'mdy' : '{d.month}-{d.day}-{d.year}',
    'mdy' : '{d.day}-{d.month}-{d.year}',
}

class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
        
    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)

In [13]:
d = Date(2017,9,12)
format(d)
print ("The date is {:ymd}".format(d))
print ("The date is {:mdy}".format(d))

The date is 2017-9-12
The date is 12-9-2017


### 8.3 객체의 콘텍스트 관리 프로토골 지원

In [18]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = AF_INET
        self.type = SOCK_STREAM
        self.sock = None
    
    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('Already connected')
        self.sock = socket(self.family, self.type)
        self.sock.connect(self.address)
        return self.sock
    
    def __exit__(self, exc_try, exc_val, tb):
        self.sock.close()
        self.sock = None

In [19]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = family
        self.type = SOCK_STREAM
        self.connections = []
        
    def __enter__(self):
        sock = socket(self.family, self.type)
        sock.connect(self.address)
        self.connections.append(sock)
        return sock
    
    def __exit__(self, exc_try, exc_val, tb):
        self.connections.pop().close()

In [20]:
# 다중 스레드에서 잘 동작 12.6
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = AF_INET
        self.type = SOCK_STREAM
        self.local = threading.local()
        
    def __enter__(self):
        if hasattr(self.local, 'sock'):
            raise RuntimeError('Already connected')
        self.local.sock = socekt(self.family, self.type)
        self.local.sock.connect(self.address)
        return self.local.sock
        
    def __exit__(self, exc_try, exc_val, tb):
        self.local.sock.close()
        del self.local.sock

### 8.4 인스턴트를 많이 생성할 때 메모리 절약

In [21]:
class Date:
    __slots__ = ['year', 'month', 'day']
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

### 8.5 클래스 이름의 캡슐화

In [22]:
class A:
    def __init__(self):
        self._internal = 0
        self.public = 1
    def public_method(self):
        pass
    def _internal_method(self):
        pass

In [24]:
class B:
    def __init__(self):
        self.__private = 0
    def __private_method(self):
        pass
    def public_method(self):
        self.__private_method()

In [29]:
class C(B):
    def __init__(self):
        super().__init__()
        self.__private = 1 # B.__private를 오버라이드 하지 않는다.
    # B.__private_method()를 오버라이드 하지 않는다.
    def __pricate_method(self):
        pass